In [1]:
# Smartvote – Clean Clustering Pipeline
# =====================================
# Dieses Notebook fuehrt alle Schritte in logisch getrennten Zellen aus:
# 0) Parameter & Setup
# 1) Daten laden + Normalisierung
# 2) Deduplication via TF‑IDF + DBSCAN
# 3) SBERT‑Embeddings & HDBSCAN‑Clustering (nur Repräsentanten)
# 4) Mapping: Duplikate erben das Cluster ihres Repräsentanten
# 5) Kennzahlen & Cluster‑Analyse
# 6) Export in eine Excel‑Datei
#
# Abhaengigkeiten (nur allgemein verfuegbare Pakete):
# pandas, numpy, scikit‑learn, sentence‑transformers, hdbscan, openpyxl

# %% [markdown]
"""
## 0 | Setup & Parameter
- Passe `DATA_PATH` bei Bedarf an.
- `DUP_EPS` bestimmt, ab welcher Cosine‐Distanz zwei Fragen als quasi identisch gelten.
- `MIN_CLUSTER_SIZE` fuer HDBSCAN kannst du hier zentral steuern.
"""

'\n## 0 | Setup & Parameter\n- Passe `DATA_PATH` bei Bedarf an.\n- `DUP_EPS` bestimmt, ab welcher Cosine‐Distanz zwei Fragen als quasi identisch gelten.\n- `MIN_CLUSTER_SIZE` fuer HDBSCAN kannst du hier zentral steuern.\n'

In [2]:
# 0 | Setup & Parameter
from pathlib import Path
import unicodedata, re, json, itertools
from collections import Counter

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from sklearn.metrics import pairwise_distances
from sentence_transformers import SentenceTransformer
import hdbscan

DATA_PATH        = Path("df_de_final.xlsx")          # Rohdaten
EXPORT_PATH      = Path("cluster_ergebnis.xlsx")      # Output‑Excel
DUP_EPS          = 0.05     # 1 – 0.95 Ähnlichkeit => Cosine‑Distanz 0.05
MIN_CLUSTER_SIZE = 2        # fuer HDBSCAN

print("Parameter geladen.")

Parameter geladen.


## 1 | Daten importieren & normalisieren
Die Funktion `norm()` vereinheitlicht Gross-/Kleinschreibung,
Satzzeichen und Unicode‐Varianten.

In [3]:
# 1.1 Daten laden
if DATA_PATH.suffix.lower() == ".csv":
    df = pd.read_csv(DATA_PATH)
else:
    df = pd.read_excel(DATA_PATH)
print("Eingelesene Zeilen:", len(df))

# 1.2 Normalisierung
_punct = re.compile(r"[^\w\s]", re.UNICODE)

def norm(text: str) -> str:
    if not isinstance(text, str):
        return ""
    t = unicodedata.normalize("NFKC", text)
    t = " ".join(t.lower().strip().split())
    t = (
        t.replace("“", '"').replace("”", '"')
         .replace("‚", "'").replace("‘", "'").replace("’", "'")
    )
    t = _punct.sub("", t)
    return " ".join(t.split())

print("Normalisiere Texte …")
df["text_norm"] = df["Frage_Text"].apply(norm)
print("Fertige Normalisierung.")

# Kennzahl 1
print("Anzahl eindeutiger normierter Texte:", df["text_norm"].nunique())



Eingelesene Zeilen: 7459
Normalisiere Texte …
Fertige Normalisierung.
Anzahl eindeutiger normierter Texte: 3116


## 2 | Deduplication
Wir verwenden TF‑IDF + DBSCAN, um identische und nahezu identische Fragen
zusammenzufassen.

In [4]:
# 2.1 TF‑IDF Matrix
print("Baue TF‑IDF Matrix …")
vectorizer = TfidfVectorizer(min_df=1)
X_tfidf    = vectorizer.fit_transform(df["text_norm"])

# 2.2 DBSCAN in Cosine‑Distanz (Radius = DUP_EPS)
print("Finde Duplikatsgruppen …")
db_dup = DBSCAN(eps=DUP_EPS, min_samples=1, metric="cosine")
df["dup_grp"] = db_dup.fit_predict(X_tfidf)

# 2.3 Repräsentant: erste Frage einer Gruppe
first_idx           = df.groupby("dup_grp").head(1).index
df["is_rep"]        = df.index.isin(first_idx)
rep_df              = df[df["is_rep"]].copy()
print("Repräsentanten:", len(rep_df), "| Duplikate:", (~df["is_rep"]).sum())

# Kennzahl 2
print("Duplikatsgruppen gesamt:", df["dup_grp"].nunique())


Baue TF‑IDF Matrix …
Finde Duplikatsgruppen …
Repräsentanten: 2940 | Duplikate: 4519
Duplikatsgruppen gesamt: 2940


## 3 | Embeddings & HDBSCAN auf den Repräsentanten

In [5]:
print("Berechne SBERT Embeddings (nur Repräsentanten) …")
model      = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
rep_df["embedding"] = list(model.encode(rep_df["text_norm"].tolist(), show_progress_bar=True))

# 3.1 HDBSCAN
print("Fuehre HDBSCAN durch …")
hdb = hdbscan.HDBSCAN(min_cluster_size=MIN_CLUSTER_SIZE, metric="euclidean")
rep_df["cluster"] = hdb.fit_predict(np.vstack(rep_df["embedding"]))

# Kennzahl 3
n_clusters = rep_df["cluster"].nunique() - (rep_df["cluster"] == -1).any()
noise_cnt  = (rep_df["cluster"] == -1).sum()
print("Cluster:", n_clusters, "| Noise:", noise_cnt)

Berechne SBERT Embeddings (nur Repräsentanten) …


Batches:   0%|          | 0/92 [00:00<?, ?it/s]

Fuehre HDBSCAN durch …


c:\Users\olive\git\EPRP\.conda\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\olive\git\EPRP\.conda\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Cluster: 209 | Noise: 1890


In [6]:
# 3.2 | Noise-Fragen exportieren
noise_df = rep_df[rep_df["cluster"] == -1].copy()
noise_df.to_pickle("noise_questions.pkl")
print(f"Noise-Fragen exportiert: {len(noise_df)} → noise_questions.pkl")

# 3.3 | Clusterzentren exportieren (für Reassignment)
import joblib, numpy as np
centers = (rep_df[rep_df["cluster"] != -1]
           .groupby("cluster")["embedding"]
           .apply(lambda x: np.mean(np.vstack(x), axis=0)))
joblib.dump(centers, "cluster_centers.pkl")
print(f"Clusterzentren gespeichert: {len(centers)} → cluster_centers.pkl")

# 3.4 | komplettes DataFrame exportieren
rep_df.to_pickle("rep_df.pkl")     # nur die Repräsentanten
print("Repräsentanten gespeichert:", len(rep_df), "Zeilen → rep_df.pkl")


Noise-Fragen exportiert: 1890 → noise_questions.pkl
Clusterzentren gespeichert: 209 → cluster_centers.pkl
Repräsentanten gespeichert: 2940 Zeilen → rep_df.pkl


## 4 | Mapping: Duplikate erben das Cluster
- Fuege Embeddings fuer alle Fragen hinzu (fuer Similarity‑Score).
- Weise jedem Duplikat das Cluster seines Repräsentanten zu.

In [7]:
print("Embeddings fuer *alle* Fragen berechnen (1‑mal, kann dauern) …")
df["embedding"] = list(model.encode(df["text_norm"].tolist(), show_progress_bar=True))

# 4.1 Cluster vom Repräsentanten auf ganze Gruppe abbilden
grp2cluster = rep_df.set_index("dup_grp")["cluster"].to_dict()
df["cluster_final"] = df["dup_grp"].map(grp2cluster)

# 4.2 Cosine‑Similarity jedes Satzes zu seinem Repräsentanten berechnen
print("Berechne Similarity …")
# Vorbereitung: Embedding Lookup nach dup_grp (Root‑Embedding)
root_emb = rep_df.set_index("dup_grp")["embedding"].to_dict()

def cos_sim(u,v):
    u = np.asarray(u); v = np.asarray(v)
    return float(np.dot(u,v) / (np.linalg.norm(u)*np.linalg.norm(v) + 1e-9))

df["sim_to_root"] = [cos_sim(e, root_emb[g]) for e, g in zip(df["embedding"], df["dup_grp"])]

# Kennzahl 4
print("Unzugeordnet (Cluster = NaN):", df["cluster_final"].isna().sum())

Embeddings fuer *alle* Fragen berechnen (1‑mal, kann dauern) …


Batches:   0%|          | 0/234 [00:00<?, ?it/s]

Berechne Similarity …
Unzugeordnet (Cluster = NaN): 0


## 5 | Analyse: Cluster‑Statistik & Top‑Woerter

In [8]:
# 5 | Clusterstatistik & Top-Wörter

# 5.1 Clustergrössen -----------------------------------------------
size_tbl = (
    df["cluster_final"]
    .value_counts()
    .rename_axis("cluster")
    .reset_index(name="n")
)
print("Clustergrössen (Top 5):")
print(size_tbl.head())

# 5.2 Top-Wörter je Cluster ----------------------------------------
print("\nBestimme Top-Wörter je Cluster …")

# einfache deutschsprachige Stop-Words (frei erweiterbar)
german_stop = [
    "und", "oder", "der", "die", "das", "ein", "eine", "einer", "eines",
    "den", "im", "mit", "von", "für", "ist", "sind", "werden", "nicht",
    "auf", "in", "an", "am", "als", "bei", "sollen", "sie", "befürworten"
]

from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

key_terms = {}

for cl, grp in rep_df.groupby("cluster"):
    if cl == -1:          # Noise überspringen
        continue

    # Vectorizer OHNE 'german'-Shortcut -> eigene Liste
    vec = TfidfVectorizer(max_features=5, stop_words=german_stop)
    X   = vec.fit_transform(grp["text_norm"])

    # TF-IDF summieren -> wichtigste Wörter
    totals = np.asarray(X.sum(axis=0)).ravel()
    top_idx = totals.argsort()[::-1][:5]
    vocab   = vec.get_feature_names_out()
    key_terms[cl] = [vocab[i] for i in top_idx]

print("\nBeispiel Top-Wörter:")
for cl, kws in list(key_terms.items())[:5]:
    print(f"Cluster {cl}: {', '.join(kws)}")


Clustergrössen (Top 5):
   cluster     n
0       -1  3951
1        7   131
2      118   111
3       61   107
4       39    99

Bestimme Top-Wörter je Cluster …

Beispiel Top-Wörter:
Cluster 0: schweizer, neuer, kampfflugzeuge, beschaffung, armee
Cluster 1: wohlfahrt, sozialversicherungen, soziale, fürsorge
Cluster 2: öffentliche, sicherheit, ordnung, verteidigung
Cluster 3: verkehr, strassenverkehr, öffentlicher
Cluster 4: nahrungsmittel, faire, eingehalten, dass, arbeitsbedingungen


## 6 | Export nach Excel
- Sheet **Clusters**: alle Fragen, nach Cluster‑Groesse sortiert; Repräsentant steht zuerst (Similarity = 1).
- Sheet **Noise**: alle Fragen mit Cluster = -1.

In [9]:
# 6.a | Hauptfrage & Ähnlichkeit pro Cluster ermitteln

# 6.a -----------------------------------------------------------------
import numpy as np

def cosine(u, v):
    u = np.asarray(u); v = np.asarray(v)
    return float(np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v) + 1e-9))

# 1. Hauptfrage (= erste Zeile mit is_rep==True) je Cluster
cluster_root_idx = {}
for cl, grp in df.groupby("cluster_final"):
    if pd.isna(cl) or cl == -1:        # Noise / unzugeordnet
        continue
    root_rows = grp[grp["is_rep"]]
    if root_rows.empty:
        root_idx = grp.index[0]        # fallback
    else:
        root_idx = root_rows.index[0]
    cluster_root_idx[cl] = root_idx

# 2. Spalten anlegen & füllen
df["ist_hauptfrage"]              = False
df["ähnlichkeit_zur_hauptfrage"]  = np.nan

for cl, root_idx in cluster_root_idx.items():
    root_emb  = df.at[root_idx, "embedding"]
    root_text = df.at[root_idx, "Frage_Text"]

    idxs = df.index[df["cluster_final"] == cl]
    df.loc[idxs, "ähnlichkeit_zur_hauptfrage"] = [
        cosine(df.at[i, "embedding"], root_emb) for i in idxs
    ]
    df.at[root_idx, "ist_hauptfrage"] = True        # Flag setzen
    df.at[root_idx, "ähnlichkeit_zur_hauptfrage"] = 1.0

# 3. Innerhalb jedes Clusters sortieren
df.sort_values(
    by=[
        "cluster_final",
        "ist_hauptfrage",                # True (=1) zuerst => absteigend sortieren
        "ähnlichkeit_zur_hauptfrage"
    ],
    ascending=[True, False, False],
    inplace=True
)

print("Spalten gefüllt – bereit für den Export.")


Spalten gefüllt – bereit für den Export.


In [10]:
# 6.b | Strukturierter Excel-Export

# %% 6.b – Export --------------------------------------------------------
import re
from collections import Counter
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill

print("Erzeuge strukturierte Excel-Datei …")

# ------------------------------------------------------------------ 6.1
export_cols = [
    "cluster_final",
    "ist_hauptfrage",
    "ähnlichkeit_zur_hauptfrage",
    "Frage_Text",
    "ID_gesamt",
]

# EINMALIGE Kopie + Altlasten entfernen
df_export = df.copy()
for old in ("is_rep", "sim_to_root"):
    if old in df_export.columns:
        df_export.drop(columns=[old], inplace=True)

# prüfen, ob Schritt 5.b lief
required = {"ist_hauptfrage", "ähnlichkeit_zur_hauptfrage"}
missing  = required - set(df_export.columns)
assert not missing, f"Fehlende Spalten {missing} – erst Schritt 5.b ausführen!"

print("Export-Spalten:", export_cols)

# Hilfsfunktion: Top-10-Nomen (Großschreib-Heuristik)
def extract_top_nouns(texts, top_n=10):
    tokens = re.findall(r"\b[A-ZÄÖÜ][a-zäöü]+\b", " ".join(map(str, texts)))
    return ", ".join(f"{w} ({c})" for w, c in Counter(tokens).most_common(top_n))

# ------------------------------------------------------------------ 6.2
unzugeordnet   = df_export[df_export["cluster_final"].isna()]
noise_clusters = df_export[df_export["cluster_final"] == -1]
regular_clusters = df_export[
    df_export["cluster_final"].notna() & (df_export["cluster_final"] != -1)
]

regular_grouped = sorted(
    regular_clusters.groupby("cluster_final"), key=lambda kv: len(kv[1]), reverse=True
)
noise_grouped = sorted(
    noise_clusters.groupby("cluster_final"), key=lambda kv: len(kv[1]), reverse=True
)

# ------------------------------------------------------------------ 6.3
wb = Workbook()
ws = wb.active
ws.title = "Smartvote Cluster"

bold_font   = Font(bold=True)
yellow_fill = PatternFill("solid", fgColor="FFFF99")

# ------------------------------------------------------------------ 6.4
def write_cluster_section(title: str, df_cluster: pd.DataFrame, start_row: int) -> int:
    ws.cell(start_row, 1, title).font = bold_font
    start_row += 1

    ws.cell(start_row, 1, f"Top 10 Nomen: {extract_top_nouns(df_cluster['Frage_Text'])}")
    start_row += 1

    # Hauptfrage zuerst, Rest nach Similarity ↓
    df_sorted = pd.concat(
        [
            df_cluster[df_cluster["ist_hauptfrage"]],
            df_cluster[~df_cluster["ist_hauptfrage"]].sort_values(
                "ähnlichkeit_zur_hauptfrage", ascending=False
            ),
        ]
    )

    # Header
    for col_idx, col in enumerate(export_cols, 1):
        ws.cell(start_row, col_idx, col).font = bold_font
    start_row += 1

    # Daten
    for _, r in df_sorted.iterrows():
        for col_idx, col in enumerate(export_cols, 1):
            val = r[col]
            cell = ws.cell(start_row, col_idx, val)
            if (
                col == "ähnlichkeit_zur_hauptfrage"
                and not r["ist_hauptfrage"]
                and val < 1.0
            ):
                # ganze Zeile gelb
                for j in range(1, len(export_cols) + 1):
                    ws.cell(start_row, j).fill = yellow_fill
        start_row += 1

    return start_row + 1  # Leerzeile

# ------------------------------------------------------------------ 6.5
row = 1
row = write_cluster_section("Unzugeordnete Fragen", unzugeordnet, row)
for cname, grp in regular_grouped:
    row = write_cluster_section(f"Cluster {cname}", grp, row)
for cname, grp in noise_grouped:
    row = write_cluster_section(cname, grp, row)

# ------------------------------------------------------------------ 6.6
wb.save(EXPORT_PATH)
print("Excel gespeichert unter:", EXPORT_PATH)


Erzeuge strukturierte Excel-Datei …
Export-Spalten: ['cluster_final', 'ist_hauptfrage', 'ähnlichkeit_zur_hauptfrage', 'Frage_Text', 'ID_gesamt']
Excel gespeichert unter: cluster_ergebnis.xlsx


In [11]:
df.to_pickle("de_final.pkl")       # jetzt mit cluster_final, sim_to_root, ...
print("Vollständiges DF gespeichert:", len(df), "Zeilen → de_final.pkl")

Vollständiges DF gespeichert: 7459 Zeilen → de_final.pkl
